Load all the csvs into dataframs and analyse.

In [51]:
import pandas as pd
from matchday_demand_forecasting.config import RAW_DATA_DIR, RANDOM_SEED

pd.set_option('display.max_columns', None)

files = ['ORDER_DETAILS.xlsx', 'ORDER_HEADER.xlsx', 'PRODUCTS.xlsx']
data_paths = [RAW_DATA_DIR / file for file in files]

df_1 = pd.read_excel(data_paths[0])
df_2 = pd.read_excel(data_paths[1])
df_3 = pd.read_excel(data_paths[2])

/opt/anaconda3/envs/matchday_demand_forecasting/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/matchday_demand_forecasting/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/matchday_demand_forecasting/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Filter table 1

In [29]:
df_1_filtered = df_1[['ORDER_HEADER_ID', 'PRODUCT_ID', 'QTY', 'SUB_TOTAL']]
df_1_filtered.head(15)

,ORDER_HEADER_ID,PRODUCT_ID,QTY,SUB_TOTAL
0,1,289,1,"8,5"
1,1,255,0,0
2,1,261,0,0
3,1,191,0,0
4,1,448,0,0
5,1,289,1,"8,5"
6,1,255,0,0
7,1,261,0,0
8,1,191,0,0
9,1,448,0,0


SUBTOTAL string and seperated by commas so first need to replace commas with point and then turn to numeric. 

In [30]:
df_1_filtered['SUB_TOTAL'] = df_1_filtered['SUB_TOTAL'].str.replace(',', '.', regex=True).astype(float)
df_1_filtered.head(15)

/var/folders/4c/4g9w277n1wb0ssy88bqbt6680000gn/T/ipykernel_49252/2254430802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_filtered['SUB_TOTAL'] = df_1_filtered['SUB_TOTAL'].str.replace(',', '.', regex=True).astype(float)


,ORDER_HEADER_ID,PRODUCT_ID,QTY,SUB_TOTAL
0,1,289,1,8.50
1,1,255,0,0.00
2,1,261,0,0.00
3,1,191,0,0.00
4,1,448,0,0.00
5,1,289,1,8.50
6,1,255,0,0.00
7,1,261,0,0.00
8,1,191,0,0.00
9,1,448,0,0.00


Filter ORDER HEADER table for only the relevant columns.

In [31]:
df_2_filtered = df_2[['ORDER_ID', 'ORDER_DATE', 'PAYMENT_METHOD']]
df_2_filtered.head()

,ORDER_ID,ORDER_DATE,PAYMENT_METHOD
0,1,12.04.2023 19:29:25,CREDIT CARD
1,2,13.04.2023 22:09:17,CREDIT CARD
2,3,13.04.2023 22:09:59,CASH
3,4,13.04.2023 22:10:54,CREDIT CARD
4,5,13.04.2023 22:13:04,CREDIT CARD


Seperate the date and time into their seperate columns

In [32]:
# Convert the column to datetime objects
df_2_filtered['ORDER_DATE'] = pd.to_datetime(df_2_filtered['ORDER_DATE'], dayfirst=True)

# Create new columns for just the date and just the time
df_2_filtered['ORDER_DATE_ONLY'] = df_2_filtered['ORDER_DATE'].dt.date
df_2_filtered['ORDER_TIME_ONLY'] = df_2_filtered['ORDER_DATE'].dt.time

df_2_filtered = df_2_filtered.drop('ORDER_DATE', axis=1)

df_2_filtered.head()

/var/folders/4c/4g9w277n1wb0ssy88bqbt6680000gn/T/ipykernel_49252/3885911971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_filtered['ORDER_DATE'] = pd.to_datetime(df_2_filtered['ORDER_DATE'], dayfirst=True)
/var/folders/4c/4g9w277n1wb0ssy88bqbt6680000gn/T/ipykernel_49252/3885911971.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_filtered['ORDER_DATE_ONLY'] = df_2_filtered['ORDER_DATE'].dt.date
/var/folders/4c/4g9w277n1wb0ssy88bqbt6680000gn/T/ipykernel_49252/3885911971.py:6: SettingWithCop

,ORDER_ID,PAYMENT_METHOD,ORDER_DATE_ONLY,ORDER_TIME_ONLY
0,1,CREDIT CARD,2023-04-12,19:29:25
1,2,CREDIT CARD,2023-04-13,22:09:17
2,3,CASH,2023-04-13,22:09:59
3,4,CREDIT CARD,2023-04-13,22:10:54
4,5,CREDIT CARD,2023-04-13,22:13:04


Join the two tables

In [34]:
df_final = pd.merge(df_1_filtered, df_2_filtered, left_on='ORDER_HEADER_ID', right_on='ORDER_ID', how='inner')
df_final = df_final.drop('ORDER_HEADER_ID', axis=1)
print(df_final.shape)
df_final.head(15)

(32097, 7)


,PRODUCT_ID,QTY,SUB_TOTAL,ORDER_ID,PAYMENT_METHOD,ORDER_DATE_ONLY,ORDER_TIME_ONLY
0,289,1,8.50,1,CREDIT CARD,2023-04-12,19:29:25
1,255,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
2,261,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
3,191,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
4,448,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
5,289,1,8.50,1,CREDIT CARD,2023-04-12,19:29:25
6,255,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
7,261,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
8,191,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25
9,448,0,0.00,1,CREDIT CARD,2023-04-12,19:29:25


Remove all the 0 quantity rows as they are not important

In [49]:
df_final = df_final[df_final['QTY'] > 0].copy()
print(df_final.shape)
df_final.head(10)

(15267, 7)


,PRODUCT_ID,QTY,SUB_TOTAL,ORDER_ID,PAYMENT_METHOD,ORDER_DATE_ONLY,ORDER_TIME_ONLY
0,289,1,8.50,1,CREDIT CARD,2023-04-12,19:29:25
5,289,1,8.50,1,CREDIT CARD,2023-04-12,19:29:25
10,329,1,6.49,2,CREDIT CARD,2023-04-13,22:09:17
11,98,1,3.99,2,CREDIT CARD,2023-04-13,22:09:17
14,339,1,6.99,3,CASH,2023-04-13,22:09:59
15,411,1,5.49,4,CREDIT CARD,2023-04-13,22:10:54
16,289,1,8.49,5,CREDIT CARD,2023-04-13,22:13:04
19,106,1,4.99,5,CREDIT CARD,2023-04-13,22:13:04
22,106,1,4.99,5,CREDIT CARD,2023-04-13,22:13:04
25,446,1,5.99,6,CREDIT CARD,2023-04-13,22:14:55
